In [9]:
import time
import pymysql
from sqlalchemy import create_engine
import sqlalchemy
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [10]:
# caps = DesiredCapabilities().CHROME
# caps["pageLoadStrategy"] = "none"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("window-size=1200x600")

In [11]:
def cody_scraping(category, wd):
    codyDf = pd.DataFrame(columns={"cody_category"})
    clothesDf = pd.DataFrame(columns=("cody_id", "clothes_maincategory", "clothes_subcategorry", "brand", "color", "season", "gender",
                                        "spring", "summer", "autumn", "winter", "img"))
    reviewDf = pd.DataFrame(columns=("clothes_id", "gender", "height", "weight", "size", "size_review", "bright", "color", "thick"))

    codyIdx = 0;
    clothesIdx = 0;
    for i in range(1, 3): #3
        print('i', i)
        url = "https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=%s&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=view_cnt&page=%d" % (category, i)
        for j in range(1, 61): #61
            wd.get(url)
            wd.find_element(By.XPATH, "/html/body/div[3]/div[3]/form/div[4]/div/ul/li[%d]/div[1]/a" % j).click()
            codyDf.loc[codyIdx] = [category]
            codyIdx += 1
            ###################################################
            # 옷 정보 크롤링
            ###################################################
            for info in range(1, 11):
                print('info', info)
                info_url = ""
                try:
                    info_url = wd.find_element(By.XPATH, '//*[@id="style_info"]/div[3]/div[2]/div/div/div[1]/div[%d]/div[1]/a'%info).get_attribute('href')
                except:
                    print(check)
                    continue

                wd.get(info_url)

                clothesMaincategory = wd.find_element(By.XPATH,'//*[@id="page_product_detail"]/div[3]/div[3]/div[1]/p/a[1]').text
                clothesSubCategory = wd.find_element(By.XPATH, '//*[@id="page_product_detail"]/div[3]/div[3]/div[1]/p/a[2]').text
                check = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[1]/h4').text
                img = wd.find_element(By.XPATH, '//*[@id="bigimg"]').get_attribute('src')

                if(check == "Product Info제품정보"):
                    brand = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]/strong/a').text
                    try:
                        season = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[1]/ul/li[2]/p[2]/strong').text
                    except:
                        season = 0
                    if(season == 0):
                        gender = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[1]/ul/li[2]/p[2]/span').text
                    else:
                        gender = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[1]/ul/li[2]/p[2]/span[2]').text
                else:
                    brand = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[2]/ul/li[1]/p[2]/strong/a').text
                    try:
                        season = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[2]/ul/li[2]/p[2]/strong').text
                    except:
                        season = 0

                    if(season == 0):
                        gender = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[2]/ul/li[2]/p[2]/span').text
                    else:
                        gender = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[2]/ul/li[2]/p[2]/span[2]').text
                # clothes_maincategory clothes_subcategorry  summer autumn spring  gender brand cody_id color season winter
                clothesDf.loc[clothesIdx] = [(codyIdx-1), clothesMaincategory, clothesSubCategory, brand, np.nan, season, gender, np.nan, np.nan, np.nan, np.nan, img]

                wd.get(info_url)
                tempDf = review_crawling(clothesIdx, wd)
                reviewDf = pd.concat([reviewDf, tempDf], ignore_index=True)
                clothesIdx += 1

                wd.back()
        # print(codyDf.tail())
    return codyDf, clothesDf, reviewDf

In [12]:
def review_crawling(clothesIdx, wd):
    reviewDf = pd.DataFrame(columns=("clothes_id", "gender", "height", "weight", "size", "size_review", "bright", "color", "thick"))
    # wd.find_element(By.XPATH, '//*[@id="estimateBox"]/div[2]/ul/li[3]').click()
    WebDriverWait(wd, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="estimateBox"]/div[2]/ul/li[3]'))).click()

    reviewIdx = 0
    last = False
    for i in range(1, 3):
        if(last):
            break
        for page in range(1, 6):
            if(last):
                break
            for idx in range(1, 12):
                if(last):
                    break

                try:
                    text = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[1]/div/div[1]/p[1]' % idx).text
                except:
                    if(idx <= 10):
                        last = True
                    else:
                        if(page == 5):
                            break
                        check = wd.find_element(By.XPATH , '//*[@id="reviewListFragment"]/div[%d]/div[2]/div/a[%d]' %  (idx, (page + 3))).text

                        if(check.isdigit() == False):
                            last = True
                            break


                        wd.execute_script(wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[2]/div/a[%d]' %  (idx, (page + 3))).get_attribute('onclick'))


                try:
                    user = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[1]/div/div[2]/p' % idx).text
                except:
                    continue

                try:
                    userArr = user.split(',')
                    gender = userArr[0]
                    height = userArr[1]
                    weight = userArr[2]
                    size = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[2]/div[2]/p/span' % idx).text
                    size_review = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[1]/span'% idx).text
                    bright = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[2]/span'% idx).text
                    color = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[3]/span'% idx).text
                    thick = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[4]/span'% idx).text
                except:
                    time.sleep(1)
                    size = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[2]/div[2]/p/span' % idx).text
                    time.sleep(1)
                    size_review = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[1]/span'% idx).text
                    time.sleep(1)
                    bright = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[2]/span'% idx).text
                    time.sleep(1)
                    color = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[3]/span'% idx).text
                    time.sleep(1)
                    thick = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[4]/span'% idx).text


                reviewDf.loc[reviewIdx] = [clothesIdx, gender, height, weight, size, size_review, bright, color, thick]
                reviewIdx += 1


        if(last == False):
            pageClick = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[8]').get_attribute('onclick')
            wd.execute_script(pageClick)
    return reviewDf



In [ ]:
wd = webdriver.Chrome('chromedriver', options=chrome_options)
wd.implicitly_wait(3)

codyDf = pd.DataFrame(columns={"cody_category"})
clothesDf = pd.DataFrame(columns=("cody_id", "clothes_maincategory", "clothes_subcategorry", "brand", "color", "season", "gender",
                                    "spring", "summer", "autumn", "winter", "img"))
reviewDf = pd.DataFrame(columns=("clothes_id", "gender", "height", "weight", "size", "size_review", "bright", "color", "thick"))
codyCategory = ["americancasual", "casual", "chic", "dandy", "formal", "girlish", "golf", "retro", "romantic", "sports", "street"]

for category in codyCategory:
    print('category', category)

    cody_temp, clothes_temp, review_temp = cody_scraping(category, wd)

    codyDf = pd.concat([codyDf, cody_temp], ignore_index=True)
    clothesDf = pd.concat([clothesDf, clothes_temp], ignore_index=True)
    reviewDf = pd.concat([reviewDf, review_temp], ignore_index=True)


category americancasual
i 1
info 1
info 2
info 3
info 4
info 5
info 6
Product Info제품정보
info 7
Product Info제품정보
info 8
Product Info제품정보
info 9
Product Info제품정보
info 10
Product Info제품정보
info 1
info 2
info 3
info 4
info 5
info 6
info 7
info 8
info 9
Product Info제품정보
info 10
Product Info제품정보
info 1
info 2
info 3
info 4
Product Info제품정보
info 5
Product Info제품정보
info 6
Product Info제품정보
info 7
Product Info제품정보
info 8
Product Info제품정보
info 9
Product Info제품정보
info 10
Product Info제품정보
info 1
info 2
Product Info제품정보
info 3
Product Info제품정보
info 4
Product Info제품정보
info 5
Product Info제품정보
info 6
Product Info제품정보
info 7
Product Info제품정보
info 8
Product Info제품정보
info 9
Product Info제품정보
info 10
Product Info제품정보
info 1
info 2
info 3
info 4
info 5
info 6
info 7
Product Info제품정보
info 8
Product Info제품정보
info 9
Product Info제품정보
info 10
Product Info제품정보
info 1
info 2
info 3
info 4
info 5
info 6
info 7
Product Info제품정보
info 8
Product Info제품정보
info 9
Product Info제품정보
info 10
Product Info제품정보
info 1
info 2
info 

In [ ]:
cody, clothes, review = cody_scraping()

In [15]:
cody

NameError: name 'cody' is not defined

In [164]:
############################################################
#리뷰 크롤링
############################################################
def review_crawling2(reviewDf, clothesIdx, wd):
    last = False
    reviewIdx = 0
    for jump in range(1, 3):
        print(jump)
        for page in range(1,6):
            lastIdx = 1
            print("page", page)
            for idx in range(1, 11):
                print('idx', idx)
                lastIdx = idx
                try:
                    text = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[1]/div/div[1]/p[1]'% idx).text
                    print(text)
                except:
                    print("여기에요")
                    if(idx < 10):
                        last = True
                    break

                # 성별, 키, 몸무게
                try:
                    user = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[1]/div/div[2]/p' % idx).text
                except:
                    continue

                user = user.split(",")
                print('user', user)
                reGender = user[0]
                height = user[1]
                weight = user[2]
                size = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[2]/div[2]/p/span' % idx).text
                sizeReview = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[1]/span' % idx).text
                bright = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[2]/span' % idx).text
                color = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[3]/span' % idx).text
                thick = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[4]/span' % idx).text

                #"clothes_id", "gender", "height", "weight", "size", "size_review", "bright", "color", "thick"
                reviewDf.loc[reviewIdx] = [clothesIdx, reGender, height, weight, size, sizeReview, bright, color, thick]
                reviewIdx += 1
            print(last)
            if(last) :
                break;
            if(lastIdx < 10):
                print('lastInx', 1, lastIdx)
                pageNum = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[2]/div/a[%d]' % ((lastIdx), (page + 3))).text
            else:
                print('lastInx', 2, lastIdx)
                pageNum = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[2]/div/a[%d]' % ((lastIdx+1), (page + 3))).text

            if(pageNum.isdigit()):
                wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[2]/div/a[%d]' % ((lastIdx+1), (page + 3))).send_keys(Keys.ENTER)
            elif (int(pageNum.isdigit()) - 1 < 8 ):
                wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[6]').send_keys(Keys.ENTER)
        print(last)
        if(last) :
            break;



    return reviewDf

In [27]:
url = 'https://www.musinsa.com/app/goods/887944/0'
wd = webdriver.Chrome('chromedriver', options=chrome_options)
wd.implicitly_wait(3)
wd.get(url)

WebDriverWait(wd, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="estimateBox"]/div[2]/ul/li[3]'))).click()

try :
    text = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[11]/div[1]/div/div[1]/p[1]').text
except:
    a = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[4]').text

    print('a', a.isdigit())

a False
